[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_stationarity/EMQA_stationarity.ipynb)

# EMQA_stationarity

Stationarity testing for oil prices and returns.

**Output:** `oil_stationarity_test.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

brent = fetch('BZ=F')
ret = np.log(brent / brent.shift(1)).dropna()

for name, series in [('Brent Price', brent), ('Brent Returns', ret)]:
    adf = adfuller(series.dropna(), autolag='AIC')
    print(f"{name}: ADF={adf[0]:.2f}, p={adf[1]:.4f}, {'Stationary' if adf[1]<0.05 else 'Non-stationary'}")

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
axes[0,0].plot(brent.index, brent, color=COLORS['blue'], lw=0.8)
axes[0,0].set_title('Prices (Non-Stationary)', fontweight='bold')
axes[0,0].set_ylabel('USD/bbl')

axes[0,1].plot(ret.index, ret, color=COLORS['red'], lw=0.5)
axes[0,1].axhline(0, color='gray', lw=0.5)
axes[0,1].set_title('Returns (Stationary)', fontweight='bold')
axes[0,1].set_ylabel('Log Return')

plot_acf(brent.dropna(), ax=axes[1,0], lags=40, alpha=0.05)
axes[1,0].set_title('ACF: Prices (slow decay)', fontweight='bold')

plot_acf(ret, ax=axes[1,1], lags=40, alpha=0.05)
axes[1,1].set_title('ACF: Returns (quick decay)', fontweight='bold')

fig.tight_layout()
save_fig(fig, 'oil_stationarity_test.pdf')
plt.show()
